In [13]:
import pandas as pd
import numpy as np
import nltk
import os
import glob
from keras.layers import SimpleRNN, Dropout, Dense, Embedding
from keras.models import Sequential
import json

In [15]:
spamFiles = glob.glob('../../data/txt_dataset/spam/*')
hamFiles = glob.glob('../../data/txt_dataset/ham/*')
wordDict = json.load(open('../../data/dictionaries/dict.json', 'r'))

In [22]:
max_words = wordDict[max(wordDict, key=lambda x: wordDict[x])]

In [63]:
bag_Of_Words_spam_binary = np.zeros(shape=(len(spamFiles), max(10000, max_words)), dtype=np.int16)
bag_Of_Words_spam = np.zeros(shape=(len(spamFiles), max(10000, max_words)), dtype=np.int16)
bag_Of_Words_ham_binary = np.zeros(shape=(len(hamFiles), max(10000, max_words)), dtype=np.int16)
bag_Of_Words_ham = np.zeros(shape=(len(hamFiles), max(10000, max_words)), dtype=np.int16)

for file_path, file_counter in zip(spamFiles, range(len(spamFiles))):
    file_data = nltk.word_tokenize(open(file_path, 'r').read().strip())
    for word in file_data:
        try:
            bag_Of_Words_spam[file_counter, wordDict[word]] += 1
            bag_Of_Words_spam_binary[file_counter, wordDict[word]] = 1
        except KeyError:
            bag_Of_Words_spam[file_counter, 0] += 1
            bag_Of_Words_spam_binary[file_counter, 0] = 1
            
for file_path, file_counter in zip(hamFiles, range(len(hamFiles))):
    file_data = nltk.word_tokenize(open(file_path, 'r').read().strip())
    for word in file_data:
        try:
            bag_Of_Words_ham[file_counter, wordDict[word]] += 1
            bag_Of_Words_ham_binary[file_counter, wordDict[word]] = 1
        except KeyError:
            bag_Of_Words_ham[file_counter, 0] += 1
            bag_Of_Words_ham_binary[file_counter, 0] = 1

In [73]:
maximum_sms_length = -np.inf
for file_path in spamFiles + hamFiles:
    maximum_sms_length = max(maximum_sms_length, len(nltk.word_tokenize(open(file_path, 'r').read().strip())))

In [80]:
message_data = np.zeros(shape = (len(spamFiles) + len(hamFiles), maximum_sms_length), dtype=np.int16)
for file_path, file_counter in zip(spamFiles + hamFiles, range(len(spamFiles) + len(hamFiles))):
    file_data = nltk.word_tokenize(open(file_path, 'r').read().strip())
    for word, word_counter in zip(file_data, range(len(file_data))):
        try:
            message_data[file_counter, word_counter] = wordDict[word]
        except KeyError:
            message_data[file_counter, word_counter] = 0

class_labels = ['spam'] * len(spamFiles) + ['ham'] * len(hamFiles)

In [86]:
class_labels = ['spam'] * len(spamFiles) + ['ham'] * len(hamFiles)

In [94]:
X, y = message_data, pd.Categorical(class_labels).codes

In [105]:
model = Sequential()
model.add(Embedding(max(10000, max_words), 10, input_length=maximum_sms_length))
model.add(SimpleRNN(10))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 171, 10)           100000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 100,221
Trainable params: 100,221
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs=10, batch_size=54)

Epoch 1/10
5574/5574 [==============================] - 4s 644us/step - loss: 0.3943 - acc: 0.8660
Epoch 2/10
5574/5574 [==============================] - 4s 784us/step - loss: 0.3943 - acc: 0.8660
Epoch 3/10
5292/5574 [===========================>..] - ETA: 0s - loss: 0.3950 - acc: 0.8655